In [ ]:
import pyautogui as gui
import cv2 as cv
import numpy as np

In [ ]:
# start and end loc
board_loc = [203, 1235, 1037, 1929]
# what's the size of the board
# TODO: detect this automatically
board_config = [6, 5]
board_size = (830, 690)
border_len = 1
# get board position here
gui.position()

In [ ]:
# based on count, get a list of every orb, this is also an ordered list
def getEachOrb(image, count):
    # weight, height
    w, h = board_size
    # column, row
    c = r = 0
    orbs = []
    if count == 20:
        # 5x4
        c = 5
        r = 4
    elif count == 30:
        # 6x5
        c = 6
        r = 5
    elif count == 42:
        # 7x6
        c = 7
        r = 6
    else:
        print("Count is not valid. Please contact the developer for more info.")

    # consider added padding here
    initial = border_len * 2
    orb_w = int((w + initial) / c)
    orb_h = int((h + initial) / r)
    x1 = y1 = initial
    x2 = y2 = initial
    # start getting every orb
    for i in range(r):
        # update y2 first
        y2 += orb_h
        for j in range(c):
            x2 += orb_w
            orbs.append(image[y1:y2, x1:x2].copy())
            # update x1 here
            x1 += orb_w
        # reset x1 and x2, update y1 here
        x1 = x2 = initial
        y1 += orb_h
    return orbs

In [ ]:
# take a screenshot at board_loc, need to subtract because it is width and height
left, top, end_left, end_top = board_loc
width = end_left - left
height = end_top - top

board_img = gui.screenshot(region=(left, top, width, height))
board_img.convert('RGB')
# save the img for open cv
board_img.save("board.png")

# scan through the entire image
# src = cv.imread("./board.png")
src = cv.imread("sample/zeus.png")

# resize it to about 830, 690 which is the size I use
src = cv.resize(src, board_size)
# add some padding in case orbs are too close to the border
src = cv.copyMakeBorder(src, border_len, border_len, border_len, border_len, cv.BORDER_CONSTANT)

# convert to RGB
image = cv.cvtColor(src, cv.COLOR_BGR2RGB)
# convert to grayscale
gray = cv.cvtColor(src, cv.COLOR_RGB2GRAY)
# create a binary thresholded image
_, binary = cv.threshold(gray, 80, 255, cv.THRESH_BINARY_INV)
# find the contours from the thresholded image
contours, _ = cv.findContours(binary, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
# This won't work for jammer (too dark) and bomb (too dark and moving) but I will detect them with image matching
goodContours = [c for c in contours if 5000 < cv.contourArea(c) < 100000]
orb_count = len(goodContours)
orbs = getEachOrb(src, orb_count)
cv.imshow("orb", orbs[11])

# draw all contours
image = cv.drawContours(src, goodContours, -1, (255, 0, 0), 7)
# cv.imshow("board", src)
# cv.imshow("bw", binary)
cv.waitKey()

In [ ]:
# save current position
(x, y) = gui.position()
offset = 500
gui.mouseDown(271, 1351, button='left')
gui.moveTo(700, 1700, duration=0.2)
gui.moveTo(500, 1000, duration=0.2)
gui.moveTo(600, 1500, duration=0.2)
gui.moveTo(200, 1300, duration=0.2)
gui.moveTo(900, 1400, duration=0.2)
gui.mouseUp()

# move back to current position after everything
gui.moveTo(x, y)